<a href="https://colab.research.google.com/github/emanbuc/LLM-playground/blob/master/TestAnthropicVertex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Software setup


```
function ClickConnect(){
    console.log("Clicked on connect button");
    document.querySelector("colab-connect-button").click()
 }
 setInterval(ClickConnect,60000)
```



In [17]:
!pip install anthropic bs4

In [18]:
!pip freeze > requirements.txt

## Setup Antrhopic Vertex

In [19]:
MODEL = "claude-3-opus@20240229"  # @param ["claude-3-sonnet@20240229", "claude-3-haiku@20240307", "claude-3-opus@20240229"]
if MODEL == "claude-3-sonnet@20240229":
    available_regions = ["us-central1", "asia-southeast1"]
elif MODEL == "claude-3-haiku@20240307":
    available_regions = ["us-central1", "europe-west4"]
else:
    available_regions = ["us-east5"]

In [20]:
import ipywidgets as widgets
from IPython.display import display

dropdown = widgets.Dropdown(
    options=available_regions,
    description="Select a region:",
    font_weight="bold",
    style={"description_width": "initial"},
)


def dropdown_eventhandler(change):
    global REGION
    if change["type"] == "change" and change["name"] == "value":
        REGION = change.new
        print("Selected:", change.new)


REGION = dropdown.value
dropdown.observe(dropdown_eventhandler, names="value")
display(dropdown)

Dropdown(description='Select a region:', options=('us-central1', 'asia-southeast1'), style=DescriptionStyle(de…

### Define Google Cloud project and model information

In [21]:
PROJECT_ID = "massimizzatore-2024-05"  # @param {type:"string"}
ENDPOINT = f"https://{REGION}-aiplatform.googleapis.com"

if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    raise ValueError("Please set your PROJECT_ID")

In [22]:
from google.colab import auth
auth.authenticate_user(project_id=PROJECT_ID)

## Test AnthropicVertex

In [23]:
from anthropic import AnthropicVertex

client = AnthropicVertex(region=REGION, project_id=PROJECT_ID)
message = client.messages.create(
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": "Send me a recipe for banana bread.",
        }
    ],
    model=MODEL,
)
print(message.model_dump_json(indent=2))

{
  "id": "msg_vrtx_01Y14XZhjHJiyFAbq4DQSdxn",
  "content": [
    {
      "text": "Here is a classic recipe for banana bread:\n\nBanana Bread\n\nIngredients:\n- 1 3/4 cups (220g) all-purpose flour\n- 1 teaspoon baking soda\n- 1/4 teaspoon salt\n- 1/2 cup (115g) unsalted butter, softened\n- 3/4 cup (150g) granulated sugar\n- 2 large eggs\n- 1 teaspoon vanilla extract\n- 1 1/4 cups (300g) mashed ripe bananas (about 3 medium bananas)\n\nInstructions:\n\n1. Preheat oven to 350°F (177°C). Grease a 9x5 inch loaf pan and set aside.\n\n2. In a medium bowl, whisk together the flour, baking soda and salt. Set aside.\n\n3. In a large bowl, beat the butter and sugar together until light and fluffy, about 2-3 minutes. Beat in the eggs one at a time, then stir in the vanilla and mashed bananas until combined. \n\n4. Fold the dry ingredients into the wet ingredients until just combined, being careful not to overmix.\n\n5. Pour the batter into the prepared loaf pan and bake for 55-65 minutes, until a 

## Cofigurazione Esprimento

In [38]:
config= {
    "massime_path": "/gdrive/MyDrive/Massime/test12",#"/gdrive/MyDrive/NomosAI/Massime",
    "sentenze_path": "/gdrive/MyDrive/Docs-Condivisi-NomosAI/Test Gold Standard", #"/gdrive/MyDrive/NomosAI/Sentenze",
    "prompt_path":"/gdrive/MyDrive/Prompt",
    "models":["claude-3-sonnet@20240229","claude-3-haiku@20240307"], #"claude-3-opus@20240229"
    "prompts":["system_message_E.txt"],#["system_message_A.txt","system_message_B.txt","system_message_C.txt"],
    "debug": False
}

## Estrazione massime da sentenze su Google Drive

In [25]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [26]:

def create_text_file_dict_from_folder(folder_path):
    import os
    import glob
    # Initialize an empty dictionary
    text_files_dict = {}

    # Create the pattern for .txt files
    pattern = os.path.join(folder_path, '*.txt')

    # Find all .txt files matching the pattern
    for file_path in glob.glob(pattern):
        # Extract the file name from the file path
        file_name = os.path.basename(file_path)

        # Read the content of the file
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()

        # Add the file name and content to the dictionary
        text_files_dict[file_name] = content

    return text_files_dict

In [27]:
def save_output_as_text_file(filename,text_content):
  percorso_file = os.path.join(config["massime_path"], filename)
  with open(percorso_file+'.txt', 'w') as f:
    f.write(text_content)

In [28]:
def genera_massima_anthropic(prompt_file,sentenza,model,output_file_name):

    prompt = prompts_files_dict[prompt_file]
    messages = [
      {'role':'user',
      'content':sentenza},
      ]

    if(config["debug"]):
      print("====== promt: ========== \n")
      print(prompt)

      print(" ======== sentenza =======\n")
      print(sentenza)


    response, token_dict = get_completion_and_token_count_anthropicAPI(messages,prompt,model=model,max_tokens= 1000)

    if(config["debug"]):
      print(" ======= response =========\n")

      print(response)

      print("======== tokens_dict ========= \n")

      print(token_dict)
    save_output_as_text_file(output_file_name+"_"+model+"_"+prompt_file,response)

In [29]:
def get_completion_and_token_count_anthropicAPI(messages,system_message,
                                   model="claude-3-sonnet@20240229",
                                   temperature=0,
                                   max_tokens=1000):


  client = AnthropicVertex(region=REGION, project_id=PROJECT_ID)
  response = client.messages.create(
        model=model,
        system=system_message,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
  )
  # print(response.model_dump_json(indent=2))

  content = response.content[0].text

  token_dict = {
    'prompt_tokens':response.usage.input_tokens,
    'completion_tokens':response.usage.output_tokens,
    'total_tokens':response.usage.input_tokens + response.usage.output_tokens
  }

  return content, token_dict


In [30]:
prompts_files_dict = create_text_file_dict_from_folder(config["prompt_path"])
for file_name, content in prompts_files_dict.items():
    print(f'File: {file_name}\n')

File: system_message_A.txt

File: system_message_B.txt

File: system_message_C.txt

File: system_message_D.txt

File: system_message_E.txt



In [40]:
from bs4 import BeautifulSoup
import time

import os
for radice, cartelle, file in os.walk(config["sentenze_path"]):
    for f in file:
      if f.endswith('.html'):
        percorso_file = os.path.join(radice, f)
        with open(percorso_file, 'r', encoding='utf-8') as fh:
          sentenza = fh.read()

          print("Elaboro file: "+ percorso_file + "\n")

          soup = BeautifulSoup(sentenza, 'html.parser')

          # Estrai il testo da tutti i tag
          testoSentenza = ''
          for paragrafo in soup.find_all('p'):
              testoSentenza += paragrafo.text + '\n'

          save_output_as_text_file(f+"_TESTO_SENTENZA",testoSentenza)

          for prompt_file in config["prompts"]:
            for model in config["models"]:
              try:
                time.sleep(30)
                genera_massima_anthropic(prompt_file,testoSentenza,model,f)
              except Exception as e:
                print("ERRORE sentenza: "+ percorso_file + " MODEL: "+model+"+ PROMPT: "+ prompt_file +"\n")
                print(f"An error occurred: {e}")

Elaboro file: /gdrive/MyDrive/Docs-Condivisi-NomosAI/Test Gold Standard/84/84_202400003_11.html

Elaboro file: /gdrive/MyDrive/Docs-Condivisi-NomosAI/Test Gold Standard/87/87_202400797_01.html

Elaboro file: /gdrive/MyDrive/Docs-Condivisi-NomosAI/Test Gold Standard/85/85_202400419_01.html

Elaboro file: /gdrive/MyDrive/Docs-Condivisi-NomosAI/Test Gold Standard/82/82_202400894_15.html

Elaboro file: /gdrive/MyDrive/Docs-Condivisi-NomosAI/Test Gold Standard/89/89_202402196_11.html

Elaboro file: /gdrive/MyDrive/Docs-Condivisi-NomosAI/Test Gold Standard/9/09_202404155_11.html

Elaboro file: /gdrive/MyDrive/Docs-Condivisi-NomosAI/Test Gold Standard/83/83_202400954_01.html

Elaboro file: /gdrive/MyDrive/Docs-Condivisi-NomosAI/Test Gold Standard/86/86_202402251_23.html

Elaboro file: /gdrive/MyDrive/Docs-Condivisi-NomosAI/Test Gold Standard/90/90_202402227_11.html

Elaboro file: /gdrive/MyDrive/Docs-Condivisi-NomosAI/Test Gold Standard/88/88_202402255_11.html

Elaboro file: /gdrive/MyDrive/D

### secondo job per Opus

In [41]:
config= {
    "massime_path": "/gdrive/MyDrive/Massime/test12",#"/gdrive/MyDrive/NomosAI/Massime",
    "sentenze_path": "/gdrive/MyDrive/Docs-Condivisi-NomosAI/Test Gold Standard", #"/gdrive/MyDrive/NomosAI/Sentenze",
    "prompt_path":"/gdrive/MyDrive/Prompt",
    "models":["claude-3-opus@20240229"], #,"claude-3-haiku@20240307"], #"claude-3-opus@20240229"
    "prompts":["system_message_E.txt"],#["system_message_A.txt","system_message_B.txt","system_message_C.txt"],
    "debug": False
}

In [42]:
MODEL = "claude-3-opus@20240229"  # @param ["claude-3-sonnet@20240229", "claude-3-haiku@20240307", "claude-3-opus@20240229"]
if MODEL == "claude-3-sonnet@20240229":
    available_regions = ["us-central1", "asia-southeast1"]
elif MODEL == "claude-3-haiku@20240307":
    available_regions = ["us-central1", "europe-west4"]
else:
    available_regions = ["us-east5"]

In [43]:
MODEL = "claude-3-opus@20240229"
REGION = "us-east5"

In [44]:
from bs4 import BeautifulSoup
import time

import os
for radice, cartelle, file in os.walk(config["sentenze_path"]):
    for f in file:
      if f.endswith('.html'):
        percorso_file = os.path.join(radice, f)
        with open(percorso_file, 'r', encoding='utf-8') as fh:
          sentenza = fh.read()

          print("Elaboro file: "+ percorso_file + "\n")

          soup = BeautifulSoup(sentenza, 'html.parser')

          # Estrai il testo da tutti i tag
          testoSentenza = ''
          for paragrafo in soup.find_all('p'):
              testoSentenza += paragrafo.text + '\n'

          save_output_as_text_file(f+"_TESTO_SENTENZA",testoSentenza)

          for prompt_file in config["prompts"]:
            for model in config["models"]:
              try:
                time.sleep(30)
                genera_massima_anthropic(prompt_file,testoSentenza,model,f)
              except Exception as e:
                print("ERRORE sentenza: "+ percorso_file + " MODEL: "+model+"+ PROMPT: "+ prompt_file +"\n")
                print(f"An error occurred: {e}")

Elaboro file: /gdrive/MyDrive/Docs-Condivisi-NomosAI/Test Gold Standard/84/84_202400003_11.html

Elaboro file: /gdrive/MyDrive/Docs-Condivisi-NomosAI/Test Gold Standard/87/87_202400797_01.html

Elaboro file: /gdrive/MyDrive/Docs-Condivisi-NomosAI/Test Gold Standard/85/85_202400419_01.html

Elaboro file: /gdrive/MyDrive/Docs-Condivisi-NomosAI/Test Gold Standard/82/82_202400894_15.html

Elaboro file: /gdrive/MyDrive/Docs-Condivisi-NomosAI/Test Gold Standard/89/89_202402196_11.html

Elaboro file: /gdrive/MyDrive/Docs-Condivisi-NomosAI/Test Gold Standard/9/09_202404155_11.html

Elaboro file: /gdrive/MyDrive/Docs-Condivisi-NomosAI/Test Gold Standard/83/83_202400954_01.html

Elaboro file: /gdrive/MyDrive/Docs-Condivisi-NomosAI/Test Gold Standard/86/86_202402251_23.html

Elaboro file: /gdrive/MyDrive/Docs-Condivisi-NomosAI/Test Gold Standard/90/90_202402227_11.html

Elaboro file: /gdrive/MyDrive/Docs-Condivisi-NomosAI/Test Gold Standard/88/88_202402255_11.html

Elaboro file: /gdrive/MyDrive/D